In [1]:
import os
import pandas as pd
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from collections import Counter
root_dir = "data/refit"
datetime_temp = "%Y-%m-%d %H:%M:%S"
pd.set_option('display.max_rows', None)
def avg_duration_evt(data_x, data_y, onthreshold, min_on = timedelta(minutes=1)):
    pre = data_x[0]
    state = "off"
    on = []
    for i,d in enumerate(data_y):
        if i > 0 and data_x[i] - data_x[i-1] > timedelta(hours=1):
            if state == "on":
                state = "off"
                this_on = data_x[i] - pre
                if this_on < min_on:
                    continue
                on.append(this_on.seconds//60)
                if data_x[i-1] - pre > timedelta(hours=10):
                    print(pre)

        if d >= onthreshold and state == "off":
            state = "on"
            pre = data_x[i]
        if d < onthreshold and state == "on":
            state = "off"
            this_on = data_x[i] - pre
            if this_on < min_on:
                continue
            on.append(this_on.seconds//60)
            if on[-1] == 1:
                print(pre)
            if data_x[i] - pre > timedelta(hours=10):
                print(pre)
    print(min(on))
    print(max(on))
    return sum(on) / (len(on) + 0.000001), on   

def get_freq_cols(df):
    cols = list(df.columns)
    freq = {}
    for c in cols:
        if "Appliance" not in c:
            continue
        freq[c] = df[c].value_counts()
    return freq

In [2]:

filename = "House1.csv"
file_path = os.path.join(root_dir, filename)
meter = pd.read_csv(file_path)
freq_cnt = get_freq_cols(meter)

y = meter["Appliance7"]
x = range(len(y))
time = [datetime.strptime(x,datetime_temp) for x in meter["Time"]]
last = 40000
# print(avg_duration_evt(time, y, 0.001))
# plt.ylim(0,0.3)


In [3]:
# print(freq_cnt["Appliance7"])
y = meter["Appliance8"]
avg, on = avg_duration_evt(time, y, 19, min_on = timedelta(minutes=5))
print(len(on))
print(avg)

2014-05-12 09:56:10
2014-07-04 12:00:19
2014-09-06 19:12:21
2014-10-24 13:03:09
2015-07-04 13:57:31
5
1032
1490
96.63087241836854


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=y,mode='markers', name='markers'))
fig.show()

In [1]:
from RefitProcessor import RefitProcessor
from config import *
from datetime import timedelta
logging.basicConfig(level=PROJ_LOGGING_LEVEL)

def prepare_refit_data():
    test_project = "refit"
    project_path = os.path.join(DATA_ROOT, test_project)
    device_list = {"House1.csv": [
                        {"name": "Appliance8", 
                        "onThreshold": 20, 
                        "minStateTime": timedelta(minutes= 5),
                        "deviceName": "TV"},
                    ],
    }
    refit_processor = RefitProcessor(project_path, {}, device_list)
    ctx_evts, device_evts = refit_processor.preprocess(output_file="p")

prepare_refit_data()

INFO:root:Load REFIT data file: House1.csv
DEBUG:root:The number of device events before filter: {'Appliance8': 7570}
('#SKIP', datetime.datetime(2013, 10, 9, 19, 11, 7))
('off', datetime.datetime(2013, 10, 9, 20, 23, 50))
('on', datetime.datetime(2013, 10, 9, 21, 17, 59))
('#SKIP', datetime.datetime(2013, 10, 26, 22, 59, 58))
('on', datetime.datetime(2013, 10, 27, 0, 0, 1))
('off', datetime.datetime(2013, 10, 27, 0, 2, 59))
('#SKIP', datetime.datetime(2013, 11, 7, 10, 3, 19))
('off', datetime.datetime(2013, 11, 8, 11, 42, 14))
('on', datetime.datetime(2013, 11, 8, 15, 35, 4))
('#SKIP', datetime.datetime(2013, 12, 16, 10, 1, 20))
('off', datetime.datetime(2013, 12, 16, 11, 6, 9))
('on', datetime.datetime(2013, 12, 16, 17, 49, 19))
('#SKIP', datetime.datetime(2013, 12, 18, 18, 18, 3))
('on', datetime.datetime(2013, 12, 18, 19, 40, 35))
('off', datetime.datetime(2013, 12, 18, 21, 1, 53))
('#SKIP', datetime.datetime(2014, 1, 15, 16, 55, 48))
('off', datetime.datetime(2014, 1, 15, 17, 49, 

IndexError: list index out of range

In [4]:
import datetime
a = datetime.datetime(2013, 10, 26, 22, 59, 58)
b = datetime.datetime(2013, 10, 27, 0, 0, 1)
